# POEM GENERATOR USING LSTM

### **Importing Required Libraries**

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras.utils as ku
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

### **Loading the Data**

In [22]:
from google.colab import files
f=files.upload()

Saving poem_1.txt to poem_1.txt


In [23]:
data = open('poem_1.txt', encoding="utf8").read()

### Creating Corpus

In [24]:
corpus = data.lower().split("\n")
print(corpus[:10])

['stay, i said', 'to the cut flowers.', 'they bowed', 'their heads lower.', 'stay, i said to the spider,', 'who fled.', 'stay, leaf.', 'it reddened,', 'embarrassed for me and itself.', 'stay, i said to my body.']


### Fitting the Tokenizer on the Corpus

In [25]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

# Vocabulary count of the corpus
total_words = len(tokenizer.word_index)

print("Total Words:", total_words)

Total Words: 3807


### **Generating Embeddings**

In [26]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]

    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences,
                                         maxlen=max_sequence_len,
                                         padding='pre'))
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = ku.to_categorical(label, num_classes=total_words+1)

### Building the LSTM Model

In [27]:
# Building a Bi-Directional LSTM Model
model = Sequential()
model.add(Embedding(total_words+1, 100,
                    input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words+1/2, activation='relu',
                kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words+1, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 100)           380800    
                                                                 
 bidirectional_1 (Bidirecti  (None, 15, 300)           301200    
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 15, 300)           0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               160400    
                                                                 
 dense_2 (Dense)             (None, 3807)              384507    
                                                                 
 dense_3 (Dense)             (None, 3808)              14500864  
                                                      

### Training the Model

In [28]:
history = model.fit(predictors, label, epochs=150, verbose=1 ,batch_size=32)

Epoch 1/150
510/510 [==============================] - 20s 29ms/step - loss: 6.9415 - accuracy: 0.0613
Epoch 2/150
510/510 [==============================] - 7s 14ms/step - loss: 6.5178 - accuracy: 0.0634
Epoch 3/150
510/510 [==============================] - 6s 12ms/step - loss: 6.3407 - accuracy: 0.0654
Epoch 4/150
510/510 [==============================] - 7s 13ms/step - loss: 6.2272 - accuracy: 0.0677
Epoch 5/150
510/510 [==============================] - 6s 11ms/step - loss: 6.1359 - accuracy: 0.0750
Epoch 6/150
510/510 [==============================] - 6s 12ms/step - loss: 6.0425 - accuracy: 0.0817
Epoch 7/150
510/510 [==============================] - 6s 11ms/step - loss: 5.9347 - accuracy: 0.0907
Epoch 8/150
510/510 [==============================] - 6s 12ms/step - loss: 5.8284 - accuracy: 0.1008
Epoch 9/150
510/510 [==============================] - 6s 11ms/step - loss: 5.7273 - accuracy: 0.1049
Epoch 10/150
510/510 [==============================] - 6s 12ms/step - loss: 5.63

### Generating Poems using the trained Model

In [41]:
print("-"*20,"Poem Generator","-"*20)
while True:
    seed_text = input("Enter a Context to Generate Poem (type 'exit' to quit): ")
    if seed_text.lower() == 'exit':
        print("-"*50)
        print("Thank You")
        break

    output_text = ""

    num_lines = int(input("Enter the number of lines: "))

    num_words_total = int(input("Enter the total number of words: "))

    print("\n")

    words_per_line = num_words_total // num_lines

    for _ in range(num_lines):
        num_words_this_line = words_per_line

        for _ in range(num_words_this_line):
            token_list = tokenizer.texts_to_sequences([seed_text])[0]
            token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
            predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
            output_word = ""

            for word, index in tokenizer.word_index.items():
                if index == predicted:
                    output_word = word
                    break

            seed_text += " " + output_word
            output_text += output_word + " "

        output_text += "\n"

    print(output_text)
    print("-"*50)

-------------------- Poem Generator --------------------
Enter a Context to Generate Poem (type 'exit' to quit): nature
Enter the number of lines: 6
Enter the total number of words: 30


is a whirl of poetry 
readings liberty than it away 
to sweet river gently stealing 
but their ra loo thyme 
a pity and shed say 
my heart day warm and 

--------------------------------------------------
Enter a Context to Generate Poem (type 'exit' to quit): emotions
Enter the number of lines: 4
Enter the total number of words: 25


i love thee to the level 
of every day's tonight girls to 
that true rich and young girls 
and cream court my heart and 

--------------------------------------------------
Enter a Context to Generate Poem (type 'exit' to quit): world 
Enter the number of lines: 5
Enter the total number of words: 25


is other and other world 
is here and there must 
no more then when the 
sea and the sheriff came 
to the mist through the 

------------------------------------------------